In [ ]:
!pip install facenet-pytorch

In [ ]:
import kagglehub

path = kagglehub.dataset_download("alfredhhw/adiencegender")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/adiencegender


In [3]:
IMAGE_DIR = "/kaggle/input/adiencegender/AdienceGender/aligned/"

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import torch
from PIL import Image
import cv2
from facenet_pytorch import MTCNN
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision.models as models
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.manifold import TSNE

In [5]:
adience_df = pd.read_csv("https://drive.google.com/uc?id=11a8GW8PXR5jYWQCb73-7IzOTYUR6abQc&export=download")

In [6]:
adience_df

,image_path,age
0,30601258@N03/landmark_aligned_face.1.103996468...,"(25, 32)"
1,30601258@N03/landmark_aligned_face.2.104248158...,"(25, 32)"
2,30601258@N03/landmark_aligned_face.1.104379798...,"(25, 32)"
3,30601258@N03/landmark_aligned_face.3.104379798...,"(25, 32)"
4,30601258@N03/landmark_aligned_face.2.118166449...,"(25, 32)"
...,...,...
13963,101515718@N03/landmark_aligned_face.2280.10587...,"(25, 32)"
13964,101515718@N03/landmark_aligned_face.2278.10587...,"(25, 32)"
13965,101515718@N03/landmark_aligned_face.2279.10587...,"(25, 32)"
13966,50458575@N08/landmark_aligned_face.2281.942669...,"(25, 32)"


In [ ]:
adience_df['image_path'] = adience_df['image_path'].astype(str)
adience_df['age'] = adience_df['age'].apply(eval)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
mtcnn = MTCNN(image_size=224, select_largest=True, keep_all=True, device=device)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomGrayscale(p=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.3, interpolation=3, fill=0),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [11]:
adience_df = adience_df[adience_df['age'] != (8, 23)]

In [12]:
sorted(adience_df["age"].unique())

[(0, 2), (4, 6), (8, 12), (15, 20), (25, 32), (38, 48), (48, 53), (60, 100)]

In [ ]:
class AdienceDataset(Dataset):

    def __init__(self, dataframe, transform=None):

        self.dataframe = dataframe

        self.transform = transform

        self.age_range_to_index = {
            age_range : idx
            for idx, age_range in enumerate(sorted(self.dataframe["age"].unique()))
        }


    def __len__(self):
        return len(self.dataframe)


    def __getitem__(self, idx):

        row = self.dataframe.iloc[idx]
        image_path = row["image_path"]

        label = self.age_range_to_index[row["age"]]
        label = torch.tensor(label, dtype=torch.long)

        image = Image.open(IMAGE_DIR + image_path)
        image = image.resize((224, 224))

        faces, _ = mtcnn.detect(image)

        if faces is None:

            image = Image.open(IMAGE_DIR + image_path)

            image = image.resize((224, 224))

        else:

            largest_face_box = faces[0]

            image = image.crop((largest_face_box[0], largest_face_box[1], largest_face_box[2], largest_face_box[3]))

        if self.transform:

            image = self.transform(image)

        return image, label

In [ ]:
dataset = AdienceDataset(adience_df, transform=transform)

train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [16]:
num_classes = 8

In [ ]:
model = models.efficientnet_b0(pretrained=True)

num_ftrs = model.classifier[-1].in_features

model.classifier[-1] = nn.Linear(num_ftrs, num_classes)

In [20]:
model = model.to(device)

In [21]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):

    model.train()

    running_loss = 0.0

    correct_train = 0
    total_train = 0

    for images, labels in train_loader:

        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item() * images.size(0)  # Total loss across all samples

        _, predicted = torch.max(outputs, 1)

        correct_train += (predicted == labels).sum().item()

        total_train += labels.size(0)

    avg_train_loss = running_loss / len(train_loader.dataset)  # Average loss per sample
    train_accuracy = 100 * correct_train / total_train

    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%")

print("Training complete.")

Epoch [1/5], Training Loss: 0.6883, Training Accuracy: 73.65%
Epoch [2/5], Training Loss: 0.5981, Training Accuracy: 77.67%
Epoch [3/5], Training Loss: 0.5277, Training Accuracy: 80.65%
Epoch [4/5], Training Loss: 0.4720, Training Accuracy: 82.39%
Epoch [5/5], Training Loss: 0.4306, Training Accuracy: 84.25%
Training complete.


In [26]:
model.eval()

correct_test = 0
total_test = 0

all_predictions = []
all_labels = []


with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)

        _, predicted = torch.max(outputs, 1)

        correct_test += (predicted == labels).sum().item()
        total_test += labels.size(0)

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


# Calculate and print the final test accuracy
test_accuracy = 100 * correct_test / total_test
print(f'Test Accuracy: {test_accuracy:.2f}%')

conf_matrix = confusion_matrix(all_labels, all_predictions)

precision, recall, f1_score, support = precision_recall_fscore_support(all_labels, all_predictions)

print("Confusion Matrix:\n", conf_matrix)
print("\nPrecision per class:", precision)
print("Recall per class:", recall)
print("F1-score per class:", f1_score)
print("Samples per class:", support)

Test Accuracy: 76.94%
Confusion Matrix:
 [[ 658   19    0    1    4    1    0    0]
 [  79  354   13    1    2    0    0    0]
 [  12   49  316   10   29    2    1    0]
 [   0    5   26  169  131    1    0    0]
 [   7    6   18    9 1279   14    5    0]
 [   2    2    3    3  297  201   21    2]
 [   1    0    0    1   46   32  111   11]
 [   0    1    0    0   17   31   37   88]]

Precision per class: [0.86693017 0.81192661 0.84042553 0.87113402 0.70858726 0.71276596
 0.63428571 0.87128713]
Recall per class: [0.96339678 0.78841871 0.75417661 0.50903614 0.95590433 0.37853107
 0.54950495 0.50574713]
F1-score per class: [0.91262136 0.8        0.79496855 0.64258555 0.8138721  0.49446494
 0.58885942 0.64      ]
Samples per class: [ 683  449  419  332 1338  531  202  174]
